# Sect 24 (v2.1): Regularization

- online-ds-pt-100719
- 03/19/2020


## Questions

- Higher level concepts
    - "Why did I just do this?"
    
- 

## Objectives

**DISCUSSION:**
- Discuss Regularization Techniques
    - Ridge Regression (L2 normalization)
    - Lasso Regression (L1 normalization)
- AIC/BIC
- Compare Feature Selection methods

**APPLICATION:**
- Practice turning repetitive code into flexible functions/loops


# Regularization techniques

We can "shrink down" prediction variables effects instead of deleting/zeroing them

## Linear Regression Cost Function Previously Used (RSS)

- ### For a single predictor (X)
$$ \large \text{cost_function}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - (mx_i + b))^2$$


- ###  multiple predictors, the equation becomes:
$$ \large \text{cost_function}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij} ) -b )^2$$
- where $k$ is the number of predictors
- and $j$ is each individual predictor.

# Penalized Estimation

> ***You've seen that when the number of predictors increases, your model complexity increases, with a higher chance of overfitting as a result.***  

> Now, instead of completely "deleting" certain predictors from a model (which is equal to setting coefficients equal to zero) we can reduce the values of the coefficients to make them less sensitive to noise in the data. 

> This is called **penalized estimation**.

> **Ridge and Lasso regression** are two examples of penalized estimation.<br>There are multiple advantages to using these methods:
- They reduce model complexity
- The may prevent from overfitting
- Some of them may perform variable selection at the same time (when coefficients are set to 0)
- They can be used to counter multicollinearity

> Lasso and Ridge are two commonly used so-called **regularization techniques**. (Regularization is a general term used when one tries to battle overfitting.)


## Ridge Regression - L2 Norm Regularization

Define a penalty ***hyperparameter*** $\lambda$ for extra terms (large $m$)

<!-- $$ =\sum_{i=1}^n(y_i - \hat{y})^2= \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij} + b))^2 + \lambda \sum_{i=1}^n m_i^2$$/-->

$$\large J_{\text{ridge}}=\sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij})-b)^2 + \lambda \sum_{j=1}^p m_j^2$$

- **By adding the penalty term $\lambda$, ridge regression puts a constraint on the coefficients $m$.**
- Therefore, large coefficients will penalize the optimization function. 
    - This shrinks the coefficients and helps to reduce model complexity and multicollinearity.

- **Error term added to cost function** 
    - $\large ...  +   \lambda \sum_{i=1}^n m_i^2$
    - Notice that  $m_i^2$ is squared, hence "L***2*** norm regularization"

- With two predictors there is a penalty term m for each predictor.
$$\large J_\text{ridge}= \sum_{i=1}^n(y_i - \hat{y})^2 = $$

$$\large  \sum_{i=1}^n(y_i - ((m_1x_{1i})-b)^2 + \lambda m_1^2 + (m_2x_{2i})-b)^2 + \lambda m_2^2)$$

### Uses

- Used mostly to prevent overfitting (but since includes all features it can be computationally expensive (for many variables))

## Lasso Regression - L1 Norm Regularization

"Least Absolute Shrinkage and Selection Operator"

$$\large J_\text{lasso}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij})-b)^2 + \lambda \sum_{j=1}^p \mid m_j \mid$$

- **Error term added to cost function** 
    - $ \large ... + \lambda \sum_{j=1}^p \mid m_j \mid$
    - Notice that  $m$ has no exponent (meaning its actually $m^1$, hence "L***1*** norm regularization"



If you have two predictors the full equation would look like this (notice that there is a penalty term `m` for each predictor in the model - in this case, two): 
$$ \text{cost_function_lasso}= \sum_{i=1}^n(y_i - \hat{y})^2 = $$

$$\sum_{i=1}^n(y_i - ((m_1x_{1i})-b)^2 + \lambda \mid m_1 \mid) + ((m_2x_{2i})-b)^2 + \lambda \mid m_2 \mid) $$

### Uses

- Lasso also helps with over fitting 
- **Lasso shrinks the less important features' coefficients to zero**, removing them altogether. 
    - Therefore, Lasso regression can be used for **feature selection**

### Using Regularization

- ***Make sure to standardize the data before performing ridge or lasso regression***, otherwise features with large values/units will be unfairly penalized.
- **Fit-transform the training data, only transform the test data**

```python
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
```

## Ridge & Lasson Regression Summary:

In Ridge regression, the cost function is changed by adding a penalty term to the square of the magnitude of the coefficients.

$$ \text{cost_function_ridge}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij})-b)^2 + \lambda \sum_{j=1}^p m_j^2$$

Lasso regression is very similar to Ridge regression, except that the magnitude of the coefficients are not squared in the penalty term.

$$ \text{cost_function_lasso}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij})-b)^2 + \lambda \sum_{j=1}^p \mid m_j \mid$$


## Applying Ridge and Lasso Regression with Scikit-Learn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-poster')

In [2]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.model_selection import train_test_split

In [3]:
## load_project data 
usecols = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'zipcode',
       'sqft_living15', 'sqft_lot15']

kc_data= 'https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project/main/data/kc_house_data.csv'
df = pd.read_csv(kc_data,usecols=usecols,na_values=['?'])
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,98178,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,98125,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,98028,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,98136,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,98074,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,3,8,1530,0.0,98103,1530,1509
21593,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,3,8,2310,0.0,98146,1830,7200
21594,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,3,7,1020,0.0,98144,1020,2007
21595,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,3,8,1600,0.0,98027,1410,1287


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21597 non-null  float64
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   sqft_lot       21597 non-null  int64  
 5   floors         21597 non-null  float64
 6   waterfront     19221 non-null  float64
 7   view           21534 non-null  float64
 8   condition      21597 non-null  int64  
 9   grade          21597 non-null  int64  
 10  sqft_above     21597 non-null  int64  
 11  sqft_basement  21143 non-null  float64
 12  zipcode        21597 non-null  int64  
 13  sqft_living15  21597 non-null  int64  
 14  sqft_lot15     21597 non-null  int64  
dtypes: float64(6), int64(9)
memory usage: 2.5 MB


In [5]:
## train test split
target = 'price'

y = df[target]
X = df.drop(target, axis=1)

# Perform test train split
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,)
#                                                      random_state=12)
[var.shape for var in [X_train , X_test, y_train, y_test]]

[(15117, 14), (6480, 14), (15117,), (6480,)]

In [6]:
## Check for nulls in training set
X_train.isna().sum()

bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       1675
view               48
condition           0
grade               0
sqft_above          0
sqft_basement     328
zipcode             0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [7]:
## Specify which values to impute with which method

## Most frequent
mode_cols = ['waterfront','view']
## Fill with 0's
zero_cols = ['sqft_basement']

In [8]:
## Copying X_train and X_test as start of X_train_tf,X_test_tf
X_train_tf = X_train.copy()
X_test_tf = X_test.copy()

In [9]:
## Impute the columns with most-frequent value
imputer_mode = SimpleImputer(strategy='most_frequent')

X_train_tf[mode_cols] = imputer_mode.fit_transform(X_train_tf[mode_cols].copy())
X_test_tf[mode_cols] = imputer_mode.transform(X_test_tf[mode_cols].copy())

## Verify it worked
X_train_tf.isna().sum()

bedrooms           0
bathrooms          0
sqft_living        0
sqft_lot           0
floors             0
waterfront         0
view               0
condition          0
grade              0
sqft_above         0
sqft_basement    328
zipcode            0
sqft_living15      0
sqft_lot15         0
dtype: int64

In [10]:
## Impute cols with 0s
imputer_zero = SimpleImputer(strategy='constant',fill_value=0)

X_train_tf[zero_cols] = imputer_zero.fit_transform(X_train_tf[zero_cols])
X_test_tf[zero_cols] = imputer_zero.transform(X_test_tf[zero_cols])

## Verify it worked
print(X_train_tf.isna().sum())
X_test_tf.isna().sum()

bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
zipcode          0
sqft_living15    0
sqft_lot15       0
dtype: int64


bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
zipcode          0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [11]:
## Specifing which cols to encode and which to scale. 
cat_cols = ['zipcode'] 
num_cols = df.drop(columns=[target,*cat_cols]).columns
num_cols

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [13]:
## Encode cat_cols
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore', drop=None)
encoder.fit(X_train_tf[cat_cols])

train_cat_cols = pd.DataFrame(encoder.transform(X_train_tf[cat_cols]),
                              columns=encoder.get_feature_names(cat_cols),
                              index=X_train_tf.index)

test_cat_cols = pd.DataFrame(encoder.transform(X_test_tf[cat_cols]),
                              columns=encoder.get_feature_names(cat_cols),
                              index=X_test_tf.index)

train_cat_cols

,zipcode_98001,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
2756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17207,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20547,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
## Scaling Num_cols
scaler = MinMaxScaler()
train_num_cols = pd.DataFrame(scaler.fit_transform(X_train_tf[num_cols]),
                              columns=num_cols,index=X_train_tf.index)

test_num_cols = pd.DataFrame(scaler.transform(X_test_tf[num_cols]),
                             columns=num_cols, index=X_test_tf.index)
train_num_cols

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15
2756,0.03125,0.200000,0.086626,0.002720,0.0,0.0,0.00,0.50,0.4,0.126246,0.000000,0.087657,0.005042
15458,0.06250,0.300000,0.228723,0.003925,0.0,0.0,0.75,0.75,0.5,0.150609,0.342324,0.205725,0.006719
6563,0.09375,0.133333,0.106383,0.002774,0.0,0.0,0.00,0.50,0.4,0.104097,0.000000,0.225403,0.005111
4404,0.06250,0.333333,0.160334,0.016466,0.4,1.0,1.00,0.50,0.7,0.233666,0.000000,0.350626,0.013173
13877,0.09375,0.266667,0.199088,0.004592,0.4,0.0,0.00,0.50,0.5,0.290144,0.000000,0.166369,0.008557
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,0.09375,0.200000,0.155775,0.120799,0.0,0.0,0.00,0.50,0.5,0.102990,0.232365,0.286225,0.171881
17207,0.09375,0.233333,0.161094,0.004834,0.4,0.0,0.00,0.75,0.5,0.234773,0.000000,0.330948,0.009705
20547,0.12500,0.366667,0.411094,0.008000,0.4,0.0,0.75,0.50,0.7,0.448505,0.282158,0.924866,0.015019
10822,0.09375,0.200000,0.151976,0.003393,0.0,0.0,0.50,0.75,0.5,0.102990,0.221992,0.212880,0.005228


In [15]:
## Combine Num and Cat Cols

X_train_tf = pd.concat([train_num_cols,train_cat_cols],axis=1)
X_test_tf = pd.concat([test_num_cols,test_cat_cols],axis=1)

X_train_tf

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
2756,0.03125,0.200000,0.086626,0.002720,0.0,0.0,0.00,0.50,0.4,0.126246,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15458,0.06250,0.300000,0.228723,0.003925,0.0,0.0,0.75,0.75,0.5,0.150609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6563,0.09375,0.133333,0.106383,0.002774,0.0,0.0,0.00,0.50,0.4,0.104097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4404,0.06250,0.333333,0.160334,0.016466,0.4,1.0,1.00,0.50,0.7,0.233666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13877,0.09375,0.266667,0.199088,0.004592,0.4,0.0,0.00,0.50,0.5,0.290144,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,0.09375,0.200000,0.155775,0.120799,0.0,0.0,0.00,0.50,0.5,0.102990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17207,0.09375,0.233333,0.161094,0.004834,0.4,0.0,0.00,0.75,0.5,0.234773,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20547,0.12500,0.366667,0.411094,0.008000,0.4,0.0,0.75,0.50,0.7,0.448505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10822,0.09375,0.200000,0.151976,0.003393,0.0,0.0,0.50,0.75,0.5,0.102990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
## confirming no values above 1
X_train_tf.max().max()

1.0000000000000002

### Exhibit A: The case for functions and dictionaries

- Code From [Ridge And Lasso Regression Lesson](https://learn.co/tracks/module-3-data-science-career-2-1/machine-learning/section-24-feature-selection-ridge-and-lasso/ridge-and-lasso-regression)

- The following code is from the Learn.co lesson and its goal is to create,fit and evaluate 3 different models to compare the results.

In [17]:
y_train.shape, y_test.shape

((15117,), (6480,))

In [18]:
# Build a Ridge, Lasso and regular linear regression model  
# Note that in scikit-learn, the regularization parameter is denoted by alpha (and not lambda)
ridge = Ridge(alpha=0.5)
ridge.fit(X_train_tf, y_train)

lasso = Lasso(alpha=0.5)
lasso.fit(X_train_tf, y_train)

lin = LinearRegression()
lin.fit(X_train_tf, y_train)


# Generate preditions for training and test sets
y_h_ridge_train = ridge.predict(X_train_tf)
y_h_ridge_test = ridge.predict(X_test_tf)

y_h_lasso_train = np.reshape(lasso.predict(X_train_tf), y_train.shape)
y_h_lasso_test = np.reshape(lasso.predict(X_test_tf),  y_test.shape)

y_h_lin_train = lin.predict(X_train_tf)
y_h_lin_test = lin.predict(X_test_tf)


print('Train Error Ridge Model', np.sum((y_train - y_h_ridge_train)**2))
print('Test Error Ridge Model', np.sum((y_test - y_h_ridge_test)**2))
print('\n')

print('Train Error Lasso Model', np.sum((y_train - y_h_lasso_train)**2))
print('Test Error Lasso Model', np.sum((y_test - y_h_lasso_test)**2))
print('\n')

print('Train Error Unpenalized Linear Model', np.sum((y_train - lin.predict(X_train_tf))**2))
print('Test Error Unpenalized Linear Model', np.sum((y_test - lin.predict(X_test_tf))**2))
print('\n'*3)
print('Ridge parameter coefficients:\n', ridge.coef_)
print('Lasso parameter coefficients:\n', lasso.coef_)
print('Linear model parameter coefficients:\n', lin.coef_)

Train Error Ridge Model 389906874192751.1
Test Error Ridge Model 179709256633007.75


Train Error Lasso Model 389821113009745.06
Test Error Lasso Model 179444839891936.2


Train Error Unpenalized Linear Model 389821815759466.0
Test Error Unpenalized Linear Model 179433570280896.0




Ridge parameter coefficients:
 [-7.31146901e+05  7.83422169e+04  1.09554082e+06  4.22841871e+05
 -1.45845004e+05  6.24553044e+05  2.30098353e+05  1.35203733e+05
  5.32805796e+05  1.19069404e+06  2.32296794e+05  7.36954124e+04
 -1.32280784e+05 -2.05322658e+05 -1.77659460e+05 -2.27631346e+05
  5.61672292e+05  9.04062013e+04  5.97846548e+04  3.84708154e+04
  3.41626548e+04 -1.44897721e+05 -9.92893602e+04 -9.11815945e+04
 -1.10446615e+05 -2.17984656e+05 -2.40262865e+05 -4.37840123e+04
 -3.78472996e+04 -7.48665490e+04  1.13277933e+03 -1.97896659e+05
 -1.95944884e+05 -1.91143958e+05  1.42529588e+05 -1.68694557e+03
 -1.76953253e+05  1.00447052e+06  3.07588633e+05 -2.02117478e+05
 -1.13326832e+05  2.11289747e+04 -

/opt/anaconda3/envs/learn-env-new/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28839243458591.312, tolerance: 198657598979.65356
  model = cd_fast.enet_coordinate_descent(


### Q1: What are the issues/limitations of the way we tested/compared the 3 different types of models? 
-  A: If we wanted to add another model type, we'd have to copy a lot and edit a lot.

### Q2: What other limitation does the above code have? (what is inflexible?)

- A: hard coded values for data shapes

### Whats the solution? 
- Functions and dictionaries!

In [19]:
from sklearn.metrics import r2_score, mean_squared_error
def make_model(model, X_train , X_test, y_train, y_test ):
    """Takes a sklearn model and train/test data to fit model, 
    get R2 and RMSE for both the training and test data.
    
    Returns:
        DataFrame: results as a df
        model: fit scikit-learn model itself"""
    
  
    model.fit(X_train, y_train)
    # Generate preditions for training and test sets
    y_hat_train = model.predict(X_train)
    y_hat_test = model.predict(X_test)
    
    r2_train = r2_score(y_train,y_hat_train)
    r2_test = r2_score(y_test,y_hat_test)
    
    rmse_train = np.sqrt(mean_squared_error(y_train,y_hat_train))
    rmse_test = np.sqrt(mean_squared_error(y_test,y_hat_test))
    
    res= [['Data','R2','RMSE']]
    res.append(['Train', r2_train, rmse_train] )
    res.append(['Test', r2_test, rmse_test] )
    
    res_df = pd.DataFrame(res[1:], columns=res[0]).round(4)
    
    return res_df, model
    

In [21]:
## Test our function
df_res, model = make_model(LinearRegression(),
                           X_train_tf, X_test_tf, y_train, y_test  )
display(df_res.round(3))

,Data,R2,RMSE
0,Train,0.804,160583.258
1,Test,0.807,166404.224


In [22]:
## Use a dictionary to store our models
models_to_make = dict( linear=LinearRegression(),
                     ridge=Ridge(alpha=0.5),
                     lasso = Lasso(alpha=0.5))

In [23]:
## Test our dictionary and function together to make a linear regression.
models_to_make['linear']

LinearRegression()

In [24]:
models_to_make

{'linear': LinearRegression(),
 'ridge': Ridge(alpha=0.5),
 'lasso': Lasso(alpha=0.5)}

In [25]:
## Loop through models_to_make and save all results

## Create an empty list to store result dfs
results = []
## Create an empty dict to store fit models
fit_models={}

for model_type,mod in models_to_make.items():

    ## get model results and fit model using make_model
    res, mod = make_model(mod, X_train_tf , X_test_tf, y_train, y_test)
    ## Add model type as column to res
    res['Type'] = model_type
    
    # Display results
    display(res)
    
    ## Save fit model to models dict 
    fit_models[model_type] = mod
    ## Save df to list 
    results.append(res)
    
## Concatenate results 
df_res = pd.concat(results)
df_res.set_index(['Type',"Data"],inplace=True)

,Data,R2,RMSE,Type
0,Train,0.8038,160583.2575,linear
1,Test,0.8066,166404.2241,linear


,Data,R2,RMSE,Type
0,Train,0.8037,160600.7760,ridge
1,Test,0.8063,166532.0089,ridge


/opt/anaconda3/envs/learn-env-new/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28839243458591.312, tolerance: 198657598979.65356
  model = cd_fast.enet_coordinate_descent(


,Data,R2,RMSE,Type
0,Train,0.8038,160583.1127,lasso
1,Test,0.8066,166409.4496,lasso


In [26]:
## Use styling to make it easier to find the best scores.
df_res.style.background_gradient(subset=['R2'])#.highlight_max('R2')

In [27]:
## Now examine the coefficients of one of the fit models
fit_models['lasso'].coef_.shape

(83,)

> #### Sidebar: You've heard of List Comprehensions but did you that there are Dictionary Comprehensions too?!

In [28]:
## Get the coefficients of every model in a dictionary using a dict comprehension
dict_comp= {k:mod.coef_ for k,mod in fit_models.items() }
dict_comp

{'linear': array([-7.93518879e+05,  7.46933087e+04,  1.30911427e+06,  4.71640203e+05,
        -1.46550758e+05,  6.27608397e+05,  2.28577755e+05,  1.35032814e+05,
         5.25220914e+05,  1.07596948e+06,  1.66936967e+05,  6.71836888e+04,
        -1.67616390e+05, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
        -1.84548198e+17, -1.84548198e+17, -1.84548198e+17, -1.84548198e+17,
  

In [29]:
## Notice the shapes of the arrays are not the same
# Print out the shapes of each dict value
for k,v in dict_comp.items():
    print(f"{k} shape = {v.shape}")
## Try making it a dataframe (error)
pd.DataFrame(dict_comp)

linear shape = (83,)
ridge shape = (83,)
lasso shape = (83,)


,linear,ridge,lasso
0,-7.935189e+05,-7.311469e+05,-7.917251e+05
1,7.469331e+04,7.834222e+04,7.421236e+04
2,1.309114e+06,1.095541e+06,1.311904e+06
3,4.716402e+05,4.228419e+05,4.695591e+05
4,-1.465508e+05,-1.458450e+05,-1.465552e+05
...,...,...,...
78,-1.845482e+17,2.436492e+04,3.447335e+04
79,-1.845482e+17,-1.499441e+05,-1.404126e+05
80,-1.845482e+17,-1.630733e+05,-1.537366e+05
81,-1.845482e+17,-1.911066e+05,-1.815600e+05


#### The solution: `.flatten()`

In [30]:
## Use .flatten() to make coefs 1D
dict_comp= {k:mod.coef_.flatten() for k,mod in fit_models.items() }

## Notice the shapes of the arrays are not the same

for k,v in dict_comp.items():
    print(f"{k} shape = {v.shape}")
## Try making it a dataframe (error)
pd.DataFrame(dict_comp)
## make it into a df

linear shape = (83,)
ridge shape = (83,)
lasso shape = (83,)


,linear,ridge,lasso
0,-7.935189e+05,-7.311469e+05,-7.917251e+05
1,7.469331e+04,7.834222e+04,7.421236e+04
2,1.309114e+06,1.095541e+06,1.311904e+06
3,4.716402e+05,4.228419e+05,4.695591e+05
4,-1.465508e+05,-1.458450e+05,-1.465552e+05
...,...,...,...
78,-1.845482e+17,2.436492e+04,3.447335e+04
79,-1.845482e+17,-1.499441e+05,-1.404126e+05
80,-1.845482e+17,-1.630733e+05,-1.537366e+05
81,-1.845482e+17,-1.911066e+05,-1.815600e+05


In [33]:
X_train_tf

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
2756,0.03125,0.200000,0.086626,0.002720,0.0,0.0,0.00,0.50,0.4,0.126246,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15458,0.06250,0.300000,0.228723,0.003925,0.0,0.0,0.75,0.75,0.5,0.150609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6563,0.09375,0.133333,0.106383,0.002774,0.0,0.0,0.00,0.50,0.4,0.104097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4404,0.06250,0.333333,0.160334,0.016466,0.4,1.0,1.00,0.50,0.7,0.233666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13877,0.09375,0.266667,0.199088,0.004592,0.4,0.0,0.00,0.50,0.5,0.290144,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,0.09375,0.200000,0.155775,0.120799,0.0,0.0,0.00,0.50,0.5,0.102990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17207,0.09375,0.233333,0.161094,0.004834,0.4,0.0,0.00,0.75,0.5,0.234773,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20547,0.12500,0.366667,0.411094,0.008000,0.4,0.0,0.75,0.50,0.7,0.448505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10822,0.09375,0.200000,0.151976,0.003393,0.0,0.0,0.50,0.75,0.5,0.102990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
### Put it all together  into 1 line of code
df_coeffs = pd.DataFrame({k:mod.coef_.flatten() for k,mod in fit_models.items()},
                        index=X_train_tf.columns)
df_coeffs

,linear,ridge,lasso
bedrooms,-7.935189e+05,-7.311469e+05,-7.917251e+05
bathrooms,7.469331e+04,7.834222e+04,7.421236e+04
sqft_living,1.309114e+06,1.095541e+06,1.311904e+06
sqft_lot,4.716402e+05,4.228419e+05,4.695591e+05
floors,-1.465508e+05,-1.458450e+05,-1.465552e+05
...,...,...,...
zipcode_98177,-1.845482e+17,2.436492e+04,3.447335e+04
zipcode_98178,-1.845482e+17,-1.499441e+05,-1.404126e+05
zipcode_98188,-1.845482e+17,-1.630733e+05,-1.537366e+05
zipcode_98198,-1.845482e+17,-1.911066e+05,-1.815600e+05


In [37]:
## Tanspose df and round
df_coeffs.T.round(2)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
linear,-793518.88,74693.31,1309114.27,471640.20,-146550.76,627608.40,228577.75,135032.81,525220.91,1075969.48,...,-1.845482e+17,-1.845482e+17,-1.845482e+17,-1.845482e+17,-1.845482e+17,-1.845482e+17,-1.845482e+17,-1.845482e+17,-1.845482e+17,-1.845482e+17
ridge,-731146.90,78342.22,1095540.82,422841.87,-145845.00,624553.04,230098.35,135203.73,532805.80,1190694.04,...,-7.850054e+04,-1.166788e+05,-5.101581e+04,-1.361666e+05,-1.133866e+05,2.436492e+04,-1.499441e+05,-1.630733e+05,-1.911066e+05,1.989706e+05
lasso,-791725.13,74212.36,1311904.37,469559.05,-146555.24,627511.27,228807.97,134962.04,525544.78,1073947.93,...,-6.895555e+04,-1.078952e+05,-4.088724e+04,-1.267611e+05,-1.041814e+05,3.447335e+04,-1.404126e+05,-1.537366e+05,-1.815600e+05,2.095637e+05


## Discussion: what was the result of the different regressions on the coefficients?

-  lasso regression shrunk some of the coefficients down to 0

# Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC)

## Uses of AIC and BIC

- Performing feature selection: comparing models with only a few variables and more variables, computing the AIC/BIC and select the features that generated the lowest AIC or BIC
- Similarly, selecting or not selecting interactions/polynomial features depending on whether or not the AIC/BIC decreases when adding them in
- Computing the AIC and BIC for several values of the regularization parameter in Ridge/Lasso models and selecting the best regularization parameter, and many more! 

### Akaike's Information Criterion (AIC)


The formula for the AIC, invented by Hirotugu Akaike in 1973 and short for "Akaike's Information Criterion" is given by:

$$ \large \text{AIC} = -2\ln(\hat{L}) + 2k $$

Where:
* $k$ : length of the parameter space (i.e. the number of features)
* $\hat{L}$ : the maximum value of the likelihood function for the model

Another way to phrase the equation is:

$$ 
\large \text{AIC(model)} =  - 2 * \text{log-likelihood(model)} + 2 * \text{length of the parameter space} $$


- AIC used to **compare** models with unbounded units not independently interpretable

- If model uses Maximum Likelihood Estimation, log-likelihood is automatically computed, so AIC is easy to calculate.
- AIC acts like penalized log-likelihood criterion, balancing good fit and complexity

- In Python, the AIC is built into `statsmodels` and in `sklearn` (such as `LassoLarsIC`, which you'll use in the upcoming lab). 

### Bayesian Information Criterion (BIC)

- Bayesian alternative to AIC, used the same way.

 $$\large \text{BIC} = -2\ln(\hat L) + \ln(n)*k $$


where:

* $\hat{L}$ and $k$ are the same as in AIC
* $n$ : the number of data points (the sample size)

Another way to phrase the equation is:

$$ \text{BIC(model)} = -2 * \text{log-likelihood(model)} + \text{log(number of observations)} * \text{(length of the parameter space)} $$

In [40]:
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
# alphas = np.arange(0.1,100,0.1)

lasso_cvA = LassoLarsIC(criterion='aic')
lasso_cvA.fit(X_train_tf,y_train);
lasso_cvA.alpha_

4.850374933197425

In [41]:
lasso_cvB = LassoLarsIC(criterion='bic')
lasso_cvB.fit(X_train_tf,y_train);
lasso_cvB.alpha_

4.850374933197425

In [42]:
def plot_ic_criterion(model, name, color):
    """Taken from:
    https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html"""
    # This is to avoid division by zero while doing np.log10
    EPSILON = 1e-4
    alpha_ = model.alpha_ + EPSILON
    alphas_ = model.alphas_ + EPSILON
    criterion_ = model.criterion_
    plt.plot(-np.log10(alphas_), criterion_, '--', color=color,
             linewidth=3, label='%s criterion' % name)
    
    plt.axvline(-np.log10(alpha_), color=color, linewidth=3,
                label=f'alpha: {name} estimate (alpha={round(alpha_,5)})')
    plt.xlabel('-log(alpha)')
    plt.ylabel('criterion')

plot_ic_criterion(lasso_cvA,'AIC','blue')
plot_ic_criterion(lasso_cvB,'BIC','green')#,'BIC' )
plt.legend(bbox_to_anchor=[1,1],loc='upper left')

# Feature Selection
[Lesson](https://learn.co/tracks/module-3-data-science-career-2-1/machine-learning/section-24-feature-selection-ridge-and-lasso/feature-selection-methods)

## Types of Feature Selection 

* Domain knowledge
* Wrapper methods
* Filter methods
* Embedded methods

> ### Wrapper Methods
<img src="https://raw.githubusercontent.com/jirvingphd/dsc-feature-selection-methods-online-ds-pt-100719/master/images/new_wrapper.png">


> ### Filter Methods
<img src="https://raw.githubusercontent.com/jirvingphd/dsc-feature-selection-methods-online-ds-pt-100719/master/images/new_filter.png">


> ### Embed Methods
<img src="https://raw.githubusercontent.com/jirvingphd/dsc-feature-selection-methods-online-ds-pt-100719/master/images/new_embedded.png">

# ACTIVITY

[**Rewrite Feature Selection Methods Lesson to easily capture and compare results across methods.**](https://learn.co/tracks/module-3-data-science-career-2-1/machine-learning/section-24-feature-selection-ridge-and-lasso/feature-selection-methods)


In [ ]:
## Feature Selection Dataset
data= "https://raw.githubusercontent.com/learn-co-students/dsc-feature-selection-methods-online-ds-pt-100719/master/diabetes.tab.txt"
df = pd.read_csv(data,sep='\t')

In [ ]:
## Make a new version of make_model that also adds a note column to results
def run_model(model,X_train,X_test,y_train,y_test,note=''):
    """Takes a sklearn model and train/test data to fit model, 
    get R2 and RMSE for both the training and test data.
    
    Returns:
        DataFrame: results as a df
        model: fit scikit-learn model itself"""
    
    res= [['Data','R2','RMSE']]
    model.fit(X_train, y_train)
    # Generate preditions for training and test sets
    y_hat_train = model.predict(X_train)
    y_hat_test = model.predict(X_test)
    
    r2_train = r2_score(y_train,y_hat_train)
    r2_test = r2_score(y_test,y_hat_test)
    
    rmse_train = np.sqrt(mean_squared_error(y_train,y_hat_train))
    rmse_test = np.sqrt(mean_squared_error(y_test,y_hat_test))
    res.append(['Train', r2_train, rmse_train] )
    res.append(['Test', r2_test, rmse_test] )
    
    res_df = pd.DataFrame(res[1:], columns=res[0]).round(4)
    res_df['Note'] = note
    return res_df#, model
    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression

# Obtain the target and features from the DataFrame
target = df['Y']
features = df.drop(columns='Y')

# Create dummy variable for sex
features['female'] = pd.get_dummies(features['SEX'], drop_first=True)
features.drop(columns=['SEX'], inplace=True)
features.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=20, test_size=0.2)
X_train

In [ ]:
# Initialize the scaler
scaler = StandardScaler()
# Scale every feature except the binary column - female
scale_cols = X_train.drop('female',axis=1).columns
scale_cols

In [ ]:
X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
X_test[scale_cols] = scaler.transform(X_test[scale_cols])

In [ ]:
## Make an empty RESULTS list
RESULTS = []
## Run our baseline regression and append to RESULTS
RESULTS.append(run_model(LinearRegression(),
                         X_train,X_test,y_train,y_test,'Baseline'))

In [ ]:
def show_results(RESULTS,style_kws={'subset':['R2']}):
    res = pd.concat(RESULTS).set_index('Note').groupby("Data").get_group('Test')   
    try:
        display(res.style.background_gradient(**style_kws))
    except:
        display(res)
show_results(RESULTS)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

X_poly_train = pd.DataFrame(poly.fit_transform(X_train),
                            columns=poly.get_feature_tfmes(features.columns))

X_poly_test = pd.DataFrame(poly.transform(X_test),
                           columns=poly.get_feature_tfmes(features.columns))

X_poly_train.head()

In [ ]:
## Make a linear regression with our X_poly data and append to RESULTS
RESULTS.append(run_model(LinearRegression(),
                         X_poly_train,X_poly_test,y_train,y_test,'Poly'))

## Show resuls
show_results(RESULTS)

### Filter Methods

>"The benefit of filter methods is that they can provide us with some useful visualizations for helping us gain an understanding about the characteristics of our data. To begin with, let's use a simple variance threshold to eliminate the features with low variance."

In [ ]:
## Filter Methods
from sklearn.feature_selection import VarianceThreshold

threshold_ranges = np.linspace(0, 2, num=6)

results = []
for thresh in threshold_ranges:
#     print(thresh)
    selector = VarianceThreshold(thresh)
    reduced_feature_train = selector.fit_transform(X_poly_train)
    reduced_feature_test = selector.transform(X_poly_test)
    lr = LinearRegression()
    lr.fit(reduced_feature_train, y_train)
    res = run_model(lr, reduced_feature_train, reduced_feature_test,
                    y_train, y_test,f"VarThresh={round(thresh,3)}")
    
    results.append(res)

res_df = pd.concat(results)
RESULTS.append(res_df)
show_results(RESULTS)

> Well, that did not seem to eliminate the features very well. It only does a little better than the base polynomial. 

In [ ]:
from sklearn.feature_selection import f_regression, mutual_info_regression, SelectKBest
selector = SelectKBest(score_func=f_regression)
X_k_best_train = selector.fit_transform(X_poly_train, y_train)
X_k_best_test= selector.transform(X_poly_test)
lr = LinearRegression()
lr.fit(X_k_best_train ,y_train)
RESULTS.append(run_model(lr,X_k_best_train,X_k_best_test,y_train,y_test,'KBest-f_regr'))

In [ ]:
selector = SelectKBest(score_func=mutual_info_regression)
X_k_best_train = selector.fit_transform(X_poly_train, y_train)
X_k_best_test= selector.transform(X_poly_test)
lr = LinearRegression()
lr.fit(X_k_best_train ,y_train)
RESULTS.append(run_model(lr,X_k_best_train,X_k_best_test,y_train,y_test,'KBest-mutual_info'))

In [ ]:
show_results(RESULTS)

> Well, that did not seem to eliminate the features very well. It only does a little better than the base polynomial. 

### Wrapper methods

> Now let's use Recursive Feature elimination (RFE) to try out a wrapper method. You'll notice that scikit-learn has a built in `RFECV()` function, which automatically determines the optimal number of features to keep when it is run based off the estimator that is passed into it. Here it is in action: 

In [ ]:
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LinearRegression

rfe = RFECV(LinearRegression(),cv=5)
X_rfe_train = rfe.fit_transform(X_poly_train, y_train)
X_rfe_test = rfe.transform(X_poly_test)
lm = LinearRegression().fit(X_rfe_train, y_train)
RESULTS.append(run_model(lm, X_rfe_train, X_rfe_test, y_train, y_test,f'RFE w. {rfe.n_features_} feats'))
# print ('The optimal number of features is: ', rfe.n_features_)

## Embedded methods  
> To compare to our other methods, we will use Lasso as the embedded method of feature selection. Luckily for us, sklearn has a built-in method to help us find the optimal features! It performs cross validation to determine the correct regularization parameter (how much to penalize our function).

In [ ]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(max_iter=100000, cv=5)
lasso.fit(X_train, y_train)
RESULTS.append(run_model(lasso,X_train, X_test, y_train, y_test,f'Lasso - X data'))
# print('The optimal alpha for the Lasso Regression is: ', lasso.alpha_)

> Let's compare this to a model with all of the polynomial features included.

In [ ]:
lasso2 = LassoCV(max_iter=100000, cv=5)
lasso2.fit(X_poly_train, y_train)

RESULTS.append(run_model(lasso2, X_poly_train, X_poly_test, y_train, y_test,f'Lasso Poly '))
# print('The optimal alpha for the Lasso Regression is: ', lasso2.alpha_)

In [ ]:
res_df = pd.concat(RESULTS)
test_res = res_df.groupby('Data').get_group('Test')
test_res.set_index(['Note']).style.highlight_max('R2')#.sort_values('MSE',ascending=True)#.groupby('Data').get_group('Test')


In [ ]:
show_results(RESULTS,style_kws=dict(subset=['RMSE'],
                                   cmap = plt.cm.get_cmap('Greens').reversed()))

> As we can see, the regularization had minimal effect on the performance of the model, but it did improve the RMSE for the test set ever so slightly! There are no set steps someone should take in order to determine the optimal feature set. In fact, now there are automated machine learning pipelines that will determine the optimal subset of features for a given problem. One of the most important and often overlooked methods of feature selection is using domain knowledge about a given area to either eliminate features or create new ones.